<a href="https://colab.research.google.com/github/oserikov/data-science-nlp/blob/master/6_%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D1%8B%D0%B9_%D0%BF%D0%B5%D1%80%D0%B5%D0%B2%D0%BE%D0%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Машинный перевод

## Apertium

In [2]:
!pip install apertium

  Created wheel for apertium: filename=apertium-0.2.3-cp36-none-any.whl size=14709 sha256=943073f9c175b2a1af0c99273b92bf4401c79fcd45908470673c9623c7864e67
  Stored in directory: /root/.cache/pip/wheels/5d/76/ee/f4ba561c990f737cdb36ab1bff6c7676bd23d4ed4cc8cef9a0
Successfully built apertium


In [0]:
import apertium

In [0]:
apertium.install_module("fra-cat")

In [0]:
apertium.append_pair_path("/usr/share/apertium/modes/")

In [9]:
apertium.translate('fra', 'cat', 'je suis étudiant')

'sóc estudiant'

## OpenNMT

In [0]:
!git clone https://github.com/OpenNMT/OpenNMT-py


In [34]:
!apt install sox libsox-dev libsox-fmt-all
!apt install libasound2-plugins libasound2-python libsox-fmt-all libsox-dev sox

!git clone https://github.com/pytorch/audio.git
%cd audio
!git checkout 301e2e9
!python setup.py install
%cd ..
!pip install -r OpenNMT-py/requirements.opt.txt
!pip install configargparse
!pip uninstall -y torchtext
!pip install git+https://github.com/pytorch/text


In [1]:
!wget -q https://raw.githubusercontent.com/ajinkyakulkarni14/TED-Multilingual-Parallel-Corpus/master/Multilingual_Parallel_Corpus/Multi_lingual_Parallel_corpus_1.zip
!unzip -q Multi_lingual_Parallel_corpus_1.zip

--2019-11-07 08:39:51--  https://raw.githubusercontent.com/ajinkyakulkarni14/TED-Multilingual-Parallel-Corpus/master/Multilingual_Parallel_Corpus/Multi_lingual_Parallel_corpus_1.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78542347 (75M) [application/zip]
Saving to: ‘Multi_lingual_Parallel_corpus_1.zip’

Multi_lingual_Paral 100%[===================>]  74.90M   156MB/s    in 0.5s    

2019-11-07 08:39:54 (156 MB/s) - ‘Multi_lingual_Parallel_corpus_1.zip’ saved [78542347/78542347]

--2019-11-07 08:39:55--  https://raw.githubusercontent.com/ajinkyakulkarni14/TED-Multilingual-Parallel-Corpus/master/Multilingual_Parallel_Corpus/Multi_lingual_Parallel_corpus_2.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 1

In [6]:
!head Multilingual_Parllel_corpus.txt

1:ar:بالنسبة لي، تبدأ هذه القصة قبل حوالي 15 عاماً مضت،
1:nl:Voor mij begint dit verhaal ongeveer 15 jaar geleden,
1:fr:Pour moi, cette histoire a commencé il y a 15 ans,
1:de:Für mich beginnt diese Geschichte vor ungefähr 15 Jahren,
1:he:עבורי, הסיפור הזה התחיל בערך לפני 15 שנים,
1:it:Per me, questa storia inizia circa 15 anni fa,
1:ru:Для меня всё началось лет 15 тому назад, когда я работал врачом
1:es:Para mí esta historia comienza hace 15 años
1:ar:بالنسبة لي، تبدأ هذه القصة قبل حوالي 15 عاماً مضت،
1:pt-br:Para mim, esta história começa cerca de 15 anos atrás,


In [0]:
!grep -P "^\d+:ru" Multilingual_Parllel_corpus.txt | \
 sed -e "s|[0-9]\+:ru:||" \
 > ru_sentences.txt

!grep -P "^\d+:fr" Multilingual_Parllel_corpus.txt | \
 sed -e "s|[0-9]\+:fr:||" \
 > fr_sentences.txt 

In [0]:
# train test val split
fr_sentences_wc_l = !wc -l fr_sentences.txt
fr_sentences_num = int(fr_sentences_wc_l[0].split()[0])

import math

train_sentences_num = math.floor(fr_sentences_num * 0.7)
test_sentences_num = math.floor(fr_sentences_num * 0.2)
val_sentences_num = fr_sentences_num - train_sentences_num - test_sentences_num

!head -{train_sentences_num} ru_sentences.txt > ru_sentences_train.txt
!head -{train_sentences_num} fr_sentences.txt > fr_sentences_train.txt

!head -{train_sentences_num+test_sentences_num} ru_sentences.txt | \
 tail -{test_sentences_num} > ru_sentences_test.txt
!head -{train_sentences_num+test_sentences_num} fr_sentences.txt | \
 tail -{test_sentences_num} > fr_sentences_test.txt

!tail -{val_sentences_num} ru_sentences.txt > ru_sentences_val.txt
!tail -{val_sentences_num} fr_sentences.txt > fr_sentences_val.txt

In [0]:
# learn bpe
!python OpenNMT-py/tools/learn_bpe.py \
 -i ru_sentences_train.txt -o ru_sentences_train.code -s 10000

!python OpenNMT-py/tools/learn_bpe.py \
 -i fr_sentences_train.txt -o fr_sentences_train.code -s 10000

# apply bpe to source sentences
!python OpenNMT-py/tools/apply_bpe.py \
 -c ru_sentences_train.code \
 -i ru_sentences_train.txt \
 -o ru_sentences_train_bpe.txt

!python OpenNMT-py/tools/apply_bpe.py \
 -c ru_sentences_train.code \
 -i ru_sentences_test.txt \
 -o ru_sentences_test_bpe.txt

!python OpenNMT-py/tools/apply_bpe.py \
 -c ru_sentences_train.code \
 -i ru_sentences_val.txt \
 -o ru_sentences_val_bpe.txt

# apply bpe to target sentences
!python OpenNMT-py/tools/apply_bpe.py \
 -c fr_sentences_train.code \
 -i fr_sentences_train.txt \
 -o fr_sentences_train_bpe.txt

!python OpenNMT-py/tools/apply_bpe.py \
 -c fr_sentences_train.code \
 -i fr_sentences_test.txt \
 -o fr_sentences_test_bpe.txt

!python OpenNMT-py/tools/apply_bpe.py \
 -c fr_sentences_train.code \
 -i fr_sentences_val.txt \
 -o fr_sentences_val_bpe.txt

In [53]:
!mkdir saved_data
!python OpenNMT-py/preprocess.py \
 -train_src ru_sentences_train_bpe.txt \
 -train_tgt fr_sentences_train_bpe.txt \
 -valid_src ru_sentences_val_bpe.txt \
 -valid_tgt fr_sentences_val_bpe.txt \
 -save_data saved_data/

mkdir: cannot create directory ‘saved_data’: File exists
[2019-11-07 09:36:17,037 INFO] Extracting features...
[2019-11-07 09:36:17,037 INFO]  * number of source features: 0.
[2019-11-07 09:36:17,037 INFO]  * number of target features: 0.
[2019-11-07 09:36:17,037 INFO] Building `Fields` object...
[2019-11-07 09:36:17,037 INFO] Building & saving training data...
[2019-11-07 09:36:17,416 INFO] Building shard 0.
[2019-11-07 09:36:26,957 INFO]  * saving 0th train data shard to saved_data/.train.0.pt.
[2019-11-07 09:36:34,162 INFO]  * tgt vocab size: 10243.
[2019-11-07 09:36:34,213 INFO]  * src vocab size: 10320.
[2019-11-07 09:36:34,271 INFO] Building & saving validation data...
[2019-11-07 09:36:34,375 INFO] Building shard 0.
[2019-11-07 09:36:35,124 INFO]  * saving 0th valid data shard to saved_data/.valid.0.pt.


In [55]:
!mkdir saved_model
!python OpenNMT-py/train.py \
 -data saved_data/ \
 -save_model saved_model/ \
 -layers 6 -rnn_size 512 -word_vec_size 512 \
 -transformer_ff 2048 \
 -heads 8 \
 -encoder_type transformer -decoder_type transformer \
 -position_encoding \
 -train_steps 200000 \
 -max_generator_batches 2 \
 -dropout 0.1 \
 -batch_size 4096 -batch_type tokens \
 -normalization tokens \
 -accum_count 2 \
 -optim adam -adam_beta2 0.998 \
 -decay_method noam \
 -warmup_steps 8000 \
 -learning_rate 2 \
 -max_grad_norm 0 \
 -param_init 0 -param_init_glorot \
 -label_smoothing 0.1 \
 -valid_steps 1000 -save_checkpoint_steps 1000 \
 -world_size 1 -gpu_rank 0

mkdir: cannot create directory ‘saved_model’: File exists
[2019-11-07 09:37:05,378 INFO]  * src vocab size = 10320
[2019-11-07 09:37:05,379 INFO]  * tgt vocab size = 10243
[2019-11-07 09:37:05,379 INFO] Building model...
[2019-11-07 09:37:14,757 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(10320, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)


In [0]:
!mkdir saved_model
!python OpenNMT-py/train.py \
 -data saved_data/ \
 -save_model saved_model/ \
 -layers 6 -rnn_size 512 -word_vec_size 512 \
 -transformer_ff 2048 \
 -heads 8 \
 -encoder_type transformer -decoder_type transformer \
 -position_encoding \
 -train_steps 200000 \
 -max_generator_batches 2 \
 -dropout 0.1 \
 -batch_size 4096 -batch_type tokens \
 -normalization tokens \
 -accum_count 2 \
 -optim adam -adam_beta2 0.998 \
 -decay_method noam \
 -warmup_steps 8000 \
 -learning_rate 2 \
 -max_grad_norm 0 \
 -param_init 0 -param_init_glorot \
 -label_smoothing 0.1 \
 -valid_steps 1000 -save_checkpoint_steps 1000 \
 -world_size 1 -gpu_rank 0

In [61]:
!python OpenNMT-py/translate.py \
 -model saved_model/_step_3000.pt \
 -src ru_sentences_val_bpe.txt \
 -output pred.txt \
 -replace_unk \
# -verbose



[2019-11-07 10:12:37,368 INFO] Translating shard 0.
Traceback (most recent call last):
  File "OpenNMT-py/translate.py", line 6, in <module>
    main()
  File "/content/OpenNMT-py/onmt/bin/translate.py", line 49, in main
    translate(opt)
  File "/content/OpenNMT-py/onmt/bin/translate.py", line 33, in translate
    attn_debug=opt.attn_debug
  File "/content/OpenNMT-py/onmt/translate/translator.py", line 351, in translate
    batch, data.src_vocabs, attn_debug
  File "/content/OpenNMT-py/onmt/translate/translator.py", line 544, in translate_batch
    return_attention=attn_debug or self.replace_unk)
  File "/content/OpenNMT-py/onmt/translate/translator.py", line 694, in _translate_batch
    batch_offset=beam._batch_offset)
  File "/content/OpenNMT-py/onmt/translate/translator.py", line 582, in _decode_and_generate
    decoder_in, memory_bank, memory_lengths=memory_lengths, step=step
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 541, in __call__
  File 

In [66]:
!paste \
 <(sed "s|@@ ||g" ru_sentences_val_bpe.txt | head -100) \
 <(sed "s|@@ ||g" pred.txt | head -100) \
 | column -t -s $'\t'

относящихся к инфицированным ВИЧ,                              qui sont infectés par le VIH,
но и о людях в группе                                          mais aussi aux gens
об употребляющих наркотики,                                    de la drogue,
о мужчинах,                                                    des hommes
о трансгендерах,                                               de la transplantation,
о мигрантах и заключённых.                                     sur le point de vue et en revenir.
И во многих частях света эта группа                            Et dans beaucoup de parties du monde,
которые особенно уязвимы.                                      qui sont particulièrement vulnérables.
Во многих частях света есть законы,                            Il y a de nombreuses parties du monde,
отражающие лучшую часть человеческой натуры.                   la meilleure partie de la nature humaine.
Эти законы относятся к людям с ВИЧ                             Ces règles du VI